In this notebook, you will find graphs representing results from simulations previously run, so you can compare your results with these

In [1]:
# imports
from leap.simulation import Simulation

In [2]:
# default simulation
simulation = Simulation()